In [ ]:
import torch
import pandas as pd
import numpy as np
from transformers import Trainer, TrainingArguments, RobertaForSequenceClassification, RobertaTokenizer, \
                         DataCollatorWithPadding, EarlyStoppingCallback,AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset, load_metric

# Load the SST-2 dataset
dataset = load_dataset('glue', 'sst2')
print("--------------------------------Loaded Dataset-------------------------------------------------------------")

# Load the RoBERTa tokenizer
#model_name = "facebook/opt-125m"
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels = 2)
print("--------------------------------Loaded tokenizer and model-------------------------------------------------------------")

#tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['sentence'], truncation=True, max_length = 512,padding='max_length')

tokenized_dataset = dataset.map(tokenize_function, batched=True)

print("--------------------------------Tokenised Data-------------------------------------------------------------")

# Load the pre-trained RoBERTaForSequenceClassification model
#model = RobertaForSequenceClassification.from_pretrained('facebook/opt-125m', num_labels=2)

# Define the training arguments

train_epochs = 1
training_args = TrainingArguments(
    output_dir='./results',                   # Output directory
    #evaluation_strategy='steps',              # Evaluation strategy
    #save_total_limit=3,                       # Number of total save checkpoints
    learning_rate=5e-6,                       # Learning rate
    per_device_train_batch_size=4,            # Batch size per device during training
    per_device_eval_batch_size=4,             # Batch size for evaluation
    num_train_epochs=train_epochs,                       # Number of training epochs
    weight_decay=0.01,                        # Weight decay
    #push_to_hub=False,                        # Whether to upload the model to the Hugging Face model hub
    logging_dir='./logs',                     # Logging directory
    logging_steps=500,                        # Logging steps
    #load_best_model_at_end=True,              # Whether to load the best model at the end of training
    #metric_for_best_model='accuracy',         # Metric for the best model
    #greater_is_better=True,                   # Whether the metric for the best model is higher is better or not
    #gradient_accumulation_steps=16,           # Number of updates steps to accumulate before backward pass
    #per_device_eval_batch_size=2,
    seed=42
    #fp16=True,
    #fp16_opt_level="O2",
    #warmup_steps=0,                           # Number of warmup steps
    #dataloader_num_workers=4,
    #label_smoothing_factor=0.1,
    #disable_tqdm=False
)

# Load the accuracy metric
metric = load_metric('accuracy')

# Define the data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
 
  # Remove ignored index (special tokens)\n",
    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    results = clf_metrics.compute(predictions=true_predictions, references=true_labels)
    return {"precision": results["overall_precision"],"recall": results["overall_recall"],"f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}
print("--------------------------------Started training-------------------------------------------------------------")


# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    data_collator=data_collator,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    compute_metrics=compute_metrics
)

# Fine-tune the model
trainer.train()
print("--------------------------------Finished training-------------------------------------------------------------")


trainer.save_model("results/finetuned-model-nlp-{}_{}_epochs".format(model_name, train_epochs))


print("--------------------------------Saved Model-------------------------------------------------------------")

print("--------------------------------Computing Metrics-------------------------------------------------------------")

def print_metrics(dataset):
    predictions, labels, _ = trainer.predict(dataset)
    #print("predictions:")
    #print(predictions)
    #print("Lables: ")
    #print(labels)
    p = predictions, labels
    k = compute_metrics(p)
    print(k)
    
print_metrics(tokenized_dataset["validation"])